### This article is learned from <a href="https://www.kaggle.com/code/cdeotte/how-to-make-spectrogram-from-eeg">how to make spectrogram from eeg</a>,I learned its code and tried to match the input of my own model(Resnet34d). Now prepare the training data.

### Import necessary libraries

In [ ]:
import pandas as pd#导入csv文件的库
import numpy as np#进行矩阵运算的库
import warnings#避免一些可以忽略的报错
warnings.filterwarnings('ignore')#filterwarnings()方法是用于设置警告过滤器的方法，它可以控制警告信息的输出方式和级别。

### Function to make spectrogram

In [ ]:
import librosa#音频处理和分析的库

#脑电图电极的位置或区域. 'Left Lower','left upper','Right Upper','RightLower'(顺时针) 
#NAMES = ['LL','LP','RP','RR']
#eeg信号采集的相对位置
FEATS = [['Fp1','F7','T3','T5','O1'],
         ['Fp1','F3','C3','P3','O1'],
         ['Fp2','F8','T4','T6','O2'],
         ['Fp2','F4','C4','P4','O2']]

#将eeg文件转成spectrogram文件
def spectrogram_from_eeg(parquet_path):#parquet_path是eeg文件的路径
    #根据路径,加载eeg的中间50秒
    eeg = pd.read_parquet(parquet_path)
    middle = len(eeg)//2-5000
    eeg = eeg.iloc[middle:middle+10000]
    
    #初始化图片大小
    img = np.zeros((256,256,4),dtype='float32')
    
    for k in range(4):
        COLS = FEATS[k]#取出FEATS第K行的特征
        
        for kk in range(4):
            #计算电势差(第kk个位置-第KK+1个位置) 
            x = eeg[COLS[kk]].values - eeg[COLS[kk+1]].values
            
            #对缺失值填充为均值
            m = np.nanmean(x)#计算非nan位置数值的平均值
            if np.isnan(x).mean()<1:#有不是缺失值的数据
                x = np.nan_to_num(x,nan=m)#将数组x中为nan值替换为均值m
            else: #np.isnan(x).mean()==1,即全是缺失值
                x[:] = 0#填充为0

            #计算音频信号的梅尔频谱特征 (n_mels,len(x)//hop_length+1)
            #y：音频信号的波形数据,sr：音频信号的采样率.
            # hop_length：帧移（每一帧之间的步长）的长度，通过将原始音频分割成多个短时帧来进行频谱计算。
            # n_fft：FFT 窗口大小，表示每个帧的长度.
            # n_mels：梅尔滤波器的数量，决定了梅尔频谱的分辨率.
            # fmin：梅尔滤波器的最低频率,fmax：梅尔滤波器的最高频率.
            # win_length：窗口函数的长度.
            mel_spec = librosa.feature.melspectrogram(y=x, sr=200, hop_length=len(x)//256,
                  n_fft=1024, n_mels=256,fmin=0, fmax=20, win_length=128)

            """
            对每个元素取以10为底的对数，得到对数功率谱矩阵.
            根据参考功率ref对对数功率谱矩阵进行平移，使得最大值等于梅尔频谱矩阵的最大值.
            截断超过width的数据,避免出现噪声或不稳定性导致的误差.取值范围为(- infty,0)
            """
            #宽度调整
            width = (mel_spec.shape[1]//32)*32
            mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max).astype(np.float32)[:,:width]

            #类似归一化的操作
            mel_spec_db = (mel_spec_db+40)/40 
            img[:,:,k] += mel_spec_db
                
        #4个时刻的差值,故取平均.
        img[:,:,k] /= 4.0
    #变成(256,256)
    img=np.mean(img,axis=2)
    return img

### make images

In [ ]:
#获取训练数据的img
train = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/train.csv')
eeg_ids = train.eeg_id.unique()

all_eegs = {}
for i,eeg_id in enumerate(eeg_ids):
    #调用函数获取img
    img = spectrogram_from_eeg(f'/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/{eeg_id}.parquet')
    #将img保存进字典
    all_eegs[eeg_id] = img
    
#将所有数据进行保存
np.save('eeg_specs',all_eegs)